# 0. Импорт

In [1]:
import gc

import warnings

warnings.filterwarnings("ignore")

In [2]:
import os
from dotenv import load_dotenv

import torch

from transformers import BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer, pipeline

from langchain_community.vectorstores import Clickhouse, ClickhouseSettings
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.llms import HuggingFacePipeline
from langchain_core.messages import HumanMessage

In [3]:
# Загрузка .env в окружение
load_dotenv()

DB_HOST = os.getenv("DB_HOST")
DB_PORT = int(os.getenv("DB_PORT"))
DB_USER = os.getenv("DB_USER")
DB_PASS = os.getenv("DB_PASS")
DB_NAME = os.getenv("DB_NAME")

# 1. Подключение к векторному хранилищу (ClickHouse)

In [4]:
# Модель 4 эмбеддингов
embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-mpnet-base-v2",
    model_kwargs={'device': 'cpu'},
    encode_kwargs={'normalize_embeddings': True},
)

# 4 L2Distance добавить "SET allow_experimental_usearch_index=1 в if self.config.index_type: или index_type="usearch",
clickhouse_settings = ClickhouseSettings(
    host=DB_HOST,
    port=DB_PORT,
    username=DB_USER,
    password=DB_PASS,
    database=DB_NAME,
    table="hac_and_rsci_publications2",
    index_type="usearch",
    metric="euclidean",
    # n_trees=100
)

vector_store = Clickhouse(
    embedding=embedding_model,
    config=clickhouse_settings
)


def format_docs(_docs):
    """Функция для слияния чанков"""
    i = 1
    s = ""

    for doc in _docs:
        s = s + f"Документ {i}:\n" + doc.page_content + "\n\n"
        i = i + 1

    return s

C:\Users\gorde\AppData\Local\Temp\ipykernel_31216\2793177234.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(


In [5]:
from typing import List
from langchain_core.documents import Document
from langchain_core.runnables import chain


@chain
def retriever(input: dict) -> List[Document]:
    """
    Функция для получения документов (чанков) из векторного хранилища и
    добавление его score в метаданные
    """
    query = input["query"]
    k = input.get("k", 10)
    where_str = input.get("where_str", None)
    _docs, scores = zip(*vector_store.similarity_search_with_relevance_scores(
        query=query,
        k=k,
        where_str=where_str,
    ))

    for doc, score in zip(_docs, scores):
        doc.metadata["score"] = score

    return _docs

In [6]:
docs = retriever.invoke({"query": "Что такое Akka", "k": 15})
docs

(Document(metadata={'category': 'Scala', 'docid': 1226679076, 'keywords': ['parallelism', 'fault tolerance'], 'page': '1', 'tech_stack': ['Scala', 'Akka'], 'title': 'Параллелизм и отказоустойчивость в обработки больших данных', 'score': 0.9152610000145984}, page_content='для решений данных проблем,связанных со временем,можно использовать akka, akka, это набор инструментов для создания параллельных, распределенных и отказоустойчивых приложений, управляемых сообщениями, для java и scala. подход akka к обработке параллелизма основан на модели actor.'),
 Document(metadata={'category': 'Scala', 'docid': 793881656, 'keywords': ['parallelism', 'fault tolerance'], 'page': '2', 'tech_stack': ['Scala', 'Akka'], 'title': 'Параллелизм и отказоустойчивость в обработки больших данных', 'score': 1.1544608214396033}, page_content='а его среда выполнения JVM позволяет создавать  высокопроизводительные системы с легким доступом к огромным экосистемам библиотек. Далее для примера использования akka будет

In [7]:
relevant_docs = [doc for doc in docs if doc.metadata["score"] < 1.2]
relevant_docs

[Document(metadata={'category': 'Scala', 'docid': 1226679076, 'keywords': ['parallelism', 'fault tolerance'], 'page': '1', 'tech_stack': ['Scala', 'Akka'], 'title': 'Параллелизм и отказоустойчивость в обработки больших данных', 'score': 0.9152610000145984}, page_content='для решений данных проблем,связанных со временем,можно использовать akka, akka, это набор инструментов для создания параллельных, распределенных и отказоустойчивых приложений, управляемых сообщениями, для java и scala. подход akka к обработке параллелизма основан на модели actor.'),
 Document(metadata={'category': 'Scala', 'docid': 793881656, 'keywords': ['parallelism', 'fault tolerance'], 'page': '2', 'tech_stack': ['Scala', 'Akka'], 'title': 'Параллелизм и отказоустойчивость в обработки больших данных', 'score': 1.1544608214396033}, page_content='а его среда выполнения JVM позволяет создавать  высокопроизводительные системы с легким доступом к огромным экосистемам библиотек. Далее для примера использования akka будет

In [8]:
retriever.invoke({"query": "Что такое Akka", "k": 15, "where_str": "metadata.category = 'Scala'"})

(Document(metadata={'category': 'Scala', 'docid': 1226679076, 'keywords': ['parallelism', 'fault tolerance'], 'page': '1', 'tech_stack': ['Scala', 'Akka'], 'title': 'Параллелизм и отказоустойчивость в обработки больших данных', 'score': 0.9152610000145984}, page_content='для решений данных проблем,связанных со временем,можно использовать akka, akka, это набор инструментов для создания параллельных, распределенных и отказоустойчивых приложений, управляемых сообщениями, для java и scala. подход akka к обработке параллелизма основан на модели actor.'),
 Document(metadata={'category': 'Scala', 'docid': 793881656, 'keywords': ['parallelism', 'fault tolerance'], 'page': '2', 'tech_stack': ['Scala', 'Akka'], 'title': 'Параллелизм и отказоустойчивость в обработки больших данных', 'score': 1.1544608214396033}, page_content='а его среда выполнения JVM позволяет создавать  высокопроизводительные системы с легким доступом к огромным экосистемам библиотек. Далее для примера использования akka будет

In [10]:
retriever.invoke({"query": "Что такое Akka", "k": 15, "where_str": "has(metadata.tech_stack, 'Python')"})

(Document(metadata={'category': 'NN', 'docid': 1414567875, 'keywords': ['computer vision'], 'page': '2', 'tech_stack': ['Python'], 'title': '«Определение жизни» с помощью машинного зрения', 'score': 1.3257382875757038}, page_content='детектор лицевых ориентиров, который реализован внутри dlib, выдает 68 точек, которые отображаются на конкретных областях лица. Эти точечные отображения были получены путем обучения предиктора на наборе данных iBUG 300-W.'),
 Document(metadata={'category': 'NN', 'docid': 2264669975, 'keywords': ['computer vision'], 'page': '1', 'tech_stack': ['Python'], 'title': '«Определение жизни» с помощью машинного зрения', 'score': 1.3267966871671832}, page_content='с использованием компьютерного зрения можно реализовать программное обеспечение, которое будет фиксировать моргание. Данное программное обеспечение можно использовать для дополнительного этапа защиты перед распознаванием лица, таким образом можно отличить снимок от настоящего человека.'),
 Document(metadat

4 rank_llm
.venv/Lib/site-packages/langchain_community/document_compressors/rankllm_rerank.py
Закомментировать в `if hasattr(rerank_results, "candidates"):`
```py
for doc in documents:
    # doc = documents[int(res.docid)]
```

# 2. Переранжирование документов (чанков)

In [6]:
from langchain_community.document_compressors.rankllm_rerank import RankLLMRerank

question = "Как определить моргания глаза"

docs = retriever.invoke({"query": question, "k": 15})

relevant_docs = [doc for doc in docs if doc.metadata["score"] < 1.2]

compressor = RankLLMRerank(top_n=3, model="zephyr")

compressed_docs = compressor.compress_documents(relevant_docs, question)

100%|██████████| 1/1 [02:22<00:00, 142.70s/it]


In [13]:
docs

(Document(metadata={'category': 'NN', 'docid': 1979324460, 'keywords': ['computer vision'], 'page': '2', 'tech_stack': ['Python'], 'title': '«Определение жизни» с помощью машинного зрения', 'score': 0.7329630317580049}, page_content='глаза описываются шестью точками. вычислив расстояние между ними, можно определить моргнул человек или нет: если состояние глаза стремится к нулю, то глаз закрыт, если состояние глаза больше заданного порога, то глаз открыт.'),
 Document(metadata={'category': 'NN', 'docid': 2264669975, 'keywords': ['computer vision'], 'page': '1', 'tech_stack': ['Python'], 'title': '«Определение жизни» с помощью машинного зрения', 'score': 0.8905457251726984}, page_content='с использованием компьютерного зрения можно реализовать программное обеспечение, которое будет фиксировать моргание. Данное программное обеспечение можно использовать для дополнительного этапа защиты перед распознаванием лица, таким образом можно отличить снимок от настоящего человека.'),
 Document(meta

In [14]:
relevant_docs

[Document(metadata={'category': 'NN', 'docid': 1979324460, 'keywords': ['computer vision'], 'page': '2', 'tech_stack': ['Python'], 'title': '«Определение жизни» с помощью машинного зрения', 'score': 0.7329630317580049}, page_content='глаза описываются шестью точками. вычислив расстояние между ними, можно определить моргнул человек или нет: если состояние глаза стремится к нулю, то глаз закрыт, если состояние глаза больше заданного порога, то глаз открыт.'),
 Document(metadata={'category': 'NN', 'docid': 2264669975, 'keywords': ['computer vision'], 'page': '1', 'tech_stack': ['Python'], 'title': '«Определение жизни» с помощью машинного зрения', 'score': 0.8905457251726984}, page_content='с использованием компьютерного зрения можно реализовать программное обеспечение, которое будет фиксировать моргание. Данное программное обеспечение можно использовать для дополнительного этапа защиты перед распознаванием лица, таким образом можно отличить снимок от настоящего человека.'),
 Document(meta

In [15]:
compressed_docs

[Document(metadata={'category': 'NN', 'docid': 1979324460, 'keywords': ['computer vision'], 'page': '2', 'tech_stack': ['Python'], 'title': '«Определение жизни» с помощью машинного зрения', 'score': 0.7329630317580049}, page_content='глаза описываются шестью точками. вычислив расстояние между ними, можно определить моргнул человек или нет: если состояние глаза стремится к нулю, то глаз закрыт, если состояние глаза больше заданного порога, то глаз открыт.'),
 Document(metadata={'category': 'NN', 'docid': 2264669975, 'keywords': ['computer vision'], 'page': '1', 'tech_stack': ['Python'], 'title': '«Определение жизни» с помощью машинного зрения', 'score': 0.8905457251726984}, page_content='с использованием компьютерного зрения можно реализовать программное обеспечение, которое будет фиксировать моргание. Данное программное обеспечение можно использовать для дополнительного этапа защиты перед распознаванием лица, таким образом можно отличить снимок от настоящего человека.'),
 Document(meta

In [7]:
# Освобождение VRAM после использования RankLLMRerank
del compressor  # Удаляем объект компрессора

gc.collect()  # Принудительный сбор мусора
torch.cuda.empty_cache()  # Очищаем кэш VRAM

# 3. LLM

In [17]:
# Настройка квантизации
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
    llm_int8_enable_fp32_cpu_offload=True,
)

# Автоматическое распределение
max_memory = {0: "6GB", "cpu": "18GB"}

# + mistralai/Mistral-7B-Instruct-v0.3              (tool)
# + mistralai/Mistral-Nemo-Instruct-2407            (12B) (NVIDIA) (длинный контекст, программирование, рус)

# + deepseek-ai/DeepSeek-V2-Lite-Chat               (16B) (MoE) (win 128к) (new, 4 чат-приложений)
# + deepseek-ai/DeepSeek-R1-Distill-Qwen-7B         (инструктивная, дистиллированная (DeepSeek-R1 (671B))) (база Qwen-2.5)
# + deepseek-ai/deepseek-coder-7b-instruct-v1.5
# deepseek-ai/deepseek-math-7b-instruct
# deepseek-ai/DeepSeek-Prover-V1.5-SFT              (SFT для точности)
# + deepseek-ai/DeepSeek-Prover-V1.5-RL             (разнообразия и адаптивности в доказательствах > SFT)
# (Supervised Fine-Tuning (SFT) на синтетических данных.
# Reinforcement Learning from Proof Assistant Feedback (RLPAF) для улучшения генерации доказательств.)

# + meta-llama/Meta-Llama-3-8B-Instruct
# meta-llama/CodeLlama-7b-hf                        (win 16k)
# + meta-llama/CodeLlama-7b-Python-hf
# + meta-llama/CodeLlama-7b-Instruct-hf
# meta-llama/Meta-Llama-Guard-2-8B                  (проверка безопасности вывода сообщения от LLM)

# CohereForAI/c4ai-command-r7b-12-2024
# CohereForAI/c4ai-command-r-plus-08-2024           (104B) (RAG, tool use)
# CohereForAI/c4ai-command-r-v01                    (35B)

# perplexity-ai/r1-1776 671B                        (MoE) (база DeepSeek-R1)
# perplexity-ai/r1-1776-distill-llama-70b 70B       (база Llama-3.3-Instruct)

# + Groq/Llama-3-Groq-8B-Tool-Use                   (4 symlinks 4 win запустить от admin python)

# Qwen/Qwen2.5-7B-Instruct
# Qwen/QwQ-32B                                      (лучше, чем DeepSeek-R1 671B, может писать на русском
# Qwen/Qwen2.5-Coder-14B-Instruct
# Qwen/Qwen2.5-Math-7B-Instruct                     (англ и китайский)
# Qwen/Qwen2.5-Math-RM-72B                          (оценка финального ответа)
# Qwen/Qwen2.5-Math-PRM-7B                          (оценка каждого шага)
# Qwen/Qwen2.5-VL-7B-Instruct                       (OCR)
# Qwen/Qwen2.5-VL-7B-Instruct-AWQ                   (OCR + < VRAM)

# + yandex/YandexGPT-5-Lite-8B-pretrain

# ai-sage/GigaChat-20B-A3B-instruct

model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-7B"
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    # trust_remote_code=True,  # Разрешаем выполнение пользовательского кода (Deepseek эксперты)
    # return_full_text=False,
    quantization_config=bnb_config,
    # "cpu": Размещает всю модель на CPU.
    # "cuda:0": Размещает всю модель на GPU с индексом 0.
    # "auto": Автоматически распределяет модель между доступными устройствами (GPU и CPU) с помощью библиотеки accelerate.
    # "balanced": Равномерно распределяет слои модели между доступными GPU.
    # "balanced_low_0": Аналогично "balanced", но с меньшей нагрузкой на устройство с индексом 0.
    # "sequential": Распределяет слои модели последовательно по доступным устройствам.
    device_map="auto",
    # low_cpu_mem_usage=True,
    torch_dtype=torch.bfloat16,
    max_memory=max_memory,
    offload_folder="D:/ws/python/llm/demo_llm_2/offload/"
)

tokenizer = AutoTokenizer.from_pretrained(model_name)
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

# Создание пайплайна для LLM
text_generation_pipeline = pipeline(
    "text-generation",
    model=model,
    model_kwargs={"use_cache": True},
    tokenizer=tokenizer,
    max_new_tokens=1024,
    temperature=0.01,  # 1, 0.7, 0.3, 0.1, 0.01, 0
    top_k=50,  # модель рассматривает только 50 токенов с наивысшей вероятностью
    top_p=0.95,  # сумм. вероятность > 95%.
    do_sample=True,
    eos_token_id=tokenizer.eos_token_id,
    repetition_penalty=1.2  # Аналог no_repeat_ngram_size
)

llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.
Loading checkpoint shards: 100%|██████████| 2/2 [00:29<00:00, 14.83s/it]
Device set to use cuda:0
C:\Users\gorde\AppData\Local\Temp\ipykernel_18604\1494585286.py:91: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=text_generation_pipeline)


In [9]:
rag_prompt = """ты агент помощник, который думает и отвечает на вопрос пользователя на русском языке и использует для этого исключительно информацию из документ из контекст. внимание!!! очень важно используй информацию из документ из контекст. не используй свои знания. за тобой наблюдают и проверяют твои ответы. поэтому перед тем как написать ответ проверь себя еще раз действительно есть ответ в документах из контекст и действительно ты написал все символы на русском языке. после проверки напиши финальный ответ
\n\nтвой алгоритм действий
\n1 запомни вопрос пользователя
\n2 изучить внимательно документ из контекст
\n3 если все документы из контекст не подходят для ответ на вопрос пользователя, то напиши [документ не найден 🤷️]
\n4 если документ из контекст подходит для ответ на вопрос пользователя, то напиши [документ найден 🕵️‍] и ответь на вопрос пользователя из информации из документ из контекст
\n\n\nизучи примеры и делай также
\n\nпример 1
\nконтекст
\nдокумент 1 mongodb, это nosql база данных, где данные хранятся в виде документов json
\nдокумент 2 mongodb используется для хранения пользовательских данных
\nвопрос пользователя
\nчто такое mongodb
\nответ
\n[документ найден 🕵️‍]
\nmongodb, это nosql база данных
\n\nпример 2
\nконтекст
\nдокумент 1 mongodb, это nosql база данных
\nвопрос пользователя
\nчто такое Hadoop
\nответ
\n[документ не найден 🤷‍]
\n\nпример 3
\nконтекст
\nдокумент 1 scala, это сильный статически типизированный высокоуровневый яп общего назначения, поддерживающий как ооп, так и фп
\nвопрос пользователя
\nкто использует api вконтакте
\nответ
\n[документ не найден 🤷‍]
\n\n\nконтекст
\n{content}
\n\nвопрос пользователя
\n{question}
\n\nдумай и отвечай только на русском языке информацией из документ из контекст. пиши кратко в 3 предложения
\nответ:"""

# 4. Run

In [10]:
question = "Как определить моргания глаза"

content = format_docs(compressed_docs)
rag_prompt_formatted = rag_prompt.format(content=content, question=question)

# result = llm.invoke([HumanMessage(content=rag_prompt_formatted)])

In [ ]:
question = "Как определить моргания глаза"

In [11]:
content

'Документ 1:\nглаза описываются шестью точками. вычислив расстояние между ними, можно определить моргнул человек или нет: если состояние глаза стремится к нулю, то глаз закрыт, если состояние глаза больше заданного порога, то глаз открыт.\n\nДокумент 2:\nс использованием компьютерного зрения можно реализовать программное обеспечение, которое будет фиксировать моргание. Данное программное обеспечение можно использовать для дополнительного этапа защиты перед распознаванием лица, таким образом можно отличить снимок от настоящего человека.\n\nДокумент 3:\nдетектор лицевых ориентиров, который реализован внутри dlib, выдает 68 точек, которые отображаются на конкретных областях лица. Эти точечные отображения были получены путем обучения предиктора на наборе данных iBUG 300-W.\n\n'

In [12]:
rag_prompt_formatted

'ты агент помощник, который думает и отвечает на вопрос пользователя на русском языке и использует для этого исключительно информацию из документ из контекст. внимание!!! очень важно используй информацию из документ из контекст. не используй свои знания. за тобой наблюдают и проверяют твои ответы. поэтому перед тем как написать ответ проверь себя еще раз действительно есть ответ в документах из контекст и действительно ты написал все символы на русском языке. после проверки напиши финальный ответ\n\n\nтвой алгоритм действий\n\n1 запомни вопрос пользователя\n\n2 изучить внимательно документ из контекст\n\n3 если все документы из контекст не подходят для ответ на вопрос пользователя, то напиши [документ не найден 🤷️]\n\n4 если документ из контекст подходит для ответ на вопрос пользователя, то напиши [документ найден 🕵️\u200d] и ответь на вопрос пользователя из информации из документ из контекст\n\n\n\nизучи примеры и делай также\n\n\nпример 1\n\nконтекст\n\nдокумент 1 mongodb, это nosql 

In [24]:
result

'Human: ты агент помощник, который думает и отвечает на вопрос пользователя на русском языке и использует для этого исключительно информацию из документ из контекст. внимание!!! очень важно используй информацию из документ из контекст. не используй свои знания. за тобой наблюдают и проверяют твои ответы. поэтому перед тем как написать ответ проверь себя еще раз действительно есть ответ в документах из контекст и действительно ты написал все символы на русском языке. после проверки напиши финальный ответ\n\n\n\nтвой алгоритм действий\n\n1 запомни вопрос пользователя\n\n2 изучить внимательно документ из контекст\n\n3 если все документы из контекст не подходят для ответ на вопрос пользователя, то напиши [документ не найден 🤷️]\n\n4 если документ из контекст подходит для ответ на вопрос пользователя, то напиши [документ найден 🕵️\u200d] и ответь на вопрос пользователя из информации из документ из контекст\n\n\n\n\nизучи примеры и делай также\n\n\nпример 1\n\nконтекст\n\nдокумент 1 mongodb,

In [28]:
print(rag_prompt_formatted)

ты агент помощник, который думает и отвечает на вопрос пользователя на русском языке и использует для этого исключительно информацию из документ из контекст. внимание!!! очень важно используй информацию из документ из контекст. не используй свои знания. за тобой наблюдают и проверяют твои ответы. поэтому перед тем как написать ответ проверь себя еще раз действительно есть ответ в документах из контекст и действительно ты написал все символы на русском языке. после проверки напиши финальный ответ



твой алгоритм действий

1 запомни вопрос пользователя

2 изучить внимательно документ из контекст

3 если все документы из контекст не подходят для ответ на вопрос пользователя, то напиши [документ не найден 🤷️]

4 если документ из контекст подходит для ответ на вопрос пользователя, то напиши [документ найден 🕵️‍] и ответь на вопрос пользователя из информации из документ из контекст




изучи примеры и делай также


пример 1

контекст

документ 1 mongodb, это nosql база данных, где данные хр

In [26]:
result[len(rag_prompt_formatted):]

'\nответ: [\n\nдокумент найден 🕵️\n\n]\n</think>\n\n[документ найден 🕵️]  \nИз предоставленного контекста видно, что моргание глаза может быть определено через анализ позиции шести точек глаз. Если расстояние между ними стремится к нулю, глаза закрыты; при значении выше определенного порога — открыты.'

In [25]:
result[result.find("</think>") + 9:]

'\n[документ найден 🕵️]  \nИз предоставленного контекста видно, что моргание глаза может быть определено через анализ позиции шести точек глаз. Если расстояние между ними стремится к нулю, глаза закрыты; при значении выше определенного порога — открыты.'

In [29]:
# Освобождение VRAM после использования DeepSeek
del model
del tokenizer
del text_generation_pipeline
del llm

gc.collect()  # Принудительный сбор мусора
torch.cuda.empty_cache()  # Очищаем кэш VRAM